In [ ]:
import random
import string
from datetime import datetime, timedelta
import pandas as pd
import mysql.connector


class OlaDataGenerator:
    def __init__(self, rows=100000):
        self.rows = rows
        self.start_date = datetime(2025, 11, 1)
        self.match_days = {5, 12, 19, 26}

        self.vehicle_types = [
            "Auto", "Prime Plus", "Prime Sedan", "Mini",
            "Bike", "eBike", "Prime SUV"
        ]

        self.payment_methods = [
            "UPI", "Cash", "Credit Card", "Debit Card", "Ola Money"
        ]

        self.pune_areas = [
            "Hinjewadi","Baner","Aundh","Wakad","Kothrud","Karve Nagar",
            "Shivaji Nagar","Deccan","Swargate","Hadapsar","Magarpatta",
            "Viman Nagar","Kharadi","Yerwada","Kalyani Nagar","Kondhwa",
            "Bibwewadi","Dhankawadi","Warje","Pashan","Balewadi",
            "Bavdhan","Sinhagad Road","Manjri","Mundhwa","Camp",
            "Fatima Nagar","Wanowrie","Nigdi","Akurdi","Chinchwad",
            "Pimpri","Bhosari","Alandi","Mosshi","Narhe","Ambegaon",
            "Katraj","Taljai","FC Road","JM Road","Model Colony",
            "Erandwane","Prabhat Road","Sadashiv Peth","Rasta Peth",
            "Kasba Peth"
        ]

    def booking_id(self):
        return "CNR" + "".join(random.choices(string.digits, k=7))

    def booking_status(self):
        return random.choices(
            ["Success", "Cancelled by Customer", "Cancelled by Driver", "Incomplete"],
            weights=[62, 7, 18, 5],
            k=1
        )[0]

    def order_value(self):
        r = random.random()
        if r <= 0.70:
            return random.randint(100, 499)
        elif r <= 0.98:
            return random.randint(500, 999)
        else:
            return random.randint(1000, 1800)

    def generate(self):
        rows = []

        for _ in range(self.rows):
            date = self.start_date + timedelta(days=random.randint(0, 29))
            status = self.booking_status()
            vehicle = random.choice(self.vehicle_types)

            row = {
                "Date": date.date(),
                "Time": f"{random.randint(0,23)}:{random.randint(0,59):02}",
                "Booking ID": self.booking_id(),
                "Booking Status": status,
                "Customer ID": f"CUST{random.randint(10000,99999)}",
                "Vehicle Type": vehicle,
                "Pickup Location": random.choice(self.pune_areas),
                "Drop Location": random.choice(self.pune_areas),
                "Avg VTAT": None,
                "Avg CTAT": None,
                "Cancelled Rides by Customer": None,
                "Reason for cancelling by Customer": None,
                "Cancelled Rides by Driver": None,
                "Incomplete Rides": None,
                "Incomplete Rides Reason": None,
                "Booking Value": None,
                "Payment Method": None,
                "Ride Distance": None,
                "Driver Ratings": None,
                "Customer Rating": None
            }

            if status == "Success":
                row["Avg VTAT"] = round(random.uniform(2, 10), 2)
                row["Avg CTAT"] = round(random.uniform(5, 15), 2)
                row["Booking Value"] = self.order_value()
                row["Ride Distance"] = round(random.uniform(1, 35), 2)
                row["Driver Ratings"] = round(random.uniform(3.5, 5.0), 1)
                row["Customer Rating"] = round(random.uniform(3.5, 5.0), 1)
                row["Payment Method"] = random.choices(
                    self.payment_methods,
                    weights=[45, 20, 15, 10, 10],
                    k=1
                )[0]

            elif status == "Cancelled by Customer":
                row["Cancelled Rides by Customer"] = 1
                row["Reason for cancelling by Customer"] = random.choice([
                    "Driver is not moving towards pickup location",
                    "Driver asked to cancel",
                    "Change of plans",
                    "Wrong Address",
                    "AC is not working" if vehicle not in ["Auto","Bike","eBike"] else "Change of plans"
                ])

            elif status == "Cancelled by Driver":
                row["Cancelled Rides by Driver"] = 1

            else:
                row["Incomplete Rides"] = 1
                row["Incomplete Rides Reason"] = random.choice(
                    ["Customer Demand", "Vehicle Breakdown", "Other Issue"]
                )

            rows.append(row)

        return pd.DataFrame(rows)

    def save_csv(self, df):
        df.to_csv("ola_bookings.csv", index=False)

    def save_mysql(self, df):
        # ✅ FORCE NaN → None
        df = df.applymap(lambda x: None if pd.isna(x) else x)

        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="12345678"  # CHANGE IF NEEDED
        )
        cursor = conn.cursor()

        cursor.execute("CREATE DATABASE IF NOT EXISTS ola_data")
        cursor.execute("USE ola_data")
        cursor.execute("DROP TABLE IF EXISTS bookings")

        cursor.execute("""
        CREATE TABLE bookings (
            `Date` DATE,
            `Time` VARCHAR(5),
            `Booking ID` VARCHAR(12),
            `Booking Status` VARCHAR(30),
            `Customer ID` VARCHAR(20),
            `Vehicle Type` VARCHAR(20),
            `Pickup Location` VARCHAR(50),
            `Drop Location` VARCHAR(50),
            `Avg VTAT` FLOAT,
            `Avg CTAT` FLOAT,
            `Cancelled Rides by Customer` INT,
            `Reason for cancelling by Customer` VARCHAR(100),
            `Cancelled Rides by Driver` INT,
            `Incomplete Rides` INT,
            `Incomplete Rides Reason` VARCHAR(50),
            `Booking Value` INT,
            `Payment Method` VARCHAR(20),
            `Ride Distance` FLOAT,
            `Driver Ratings` FLOAT,
            `Customer Rating` FLOAT
        )
        """)

        columns = ",".join(f"`{c}`" for c in df.columns)
        placeholders = ",".join(["%s"] * len(df.columns))
        insert_query = f"INSERT INTO bookings ({columns}) VALUES ({placeholders})"

        data = [tuple(row) for row in df.itertuples(index=False, name=None)]

        BATCH_SIZE = 1000
        total = len(data)

        for i in range(0, total, BATCH_SIZE):
            cursor.executemany(insert_query, data[i:i+BATCH_SIZE])
            conn.commit()
            print(f"Inserted {min(i+BATCH_SIZE, total)} / {total}")

        cursor.close()
        conn.close()


if __name__ == "__main__":
    generator = OlaDataGenerator(rows=100000)

    df = generator.generate()
    print(df.head())
    print(df.shape)

    generator.save_csv(df)
    generator.save_mysql(df)
